In [117]:
import json
import string
import random
import nltk
import datetime

nltk.download("omw-1.4")
import numpy as num
from nltk.stem import WordNetLemmatizer  # It has the ability to lemmatize.
import tensorflow as tensorF  # A multidimensional array of elements is represented by this symbol.
from tensorflow.keras import (
    Sequential,
)  # Sequential groups a linear stack of layers into a tf.keras.Model
from tensorflow.keras.layers import Dense, Dropout

nltk.download("punkt")  # required package for tokenization
nltk.download("wordnet")  # word database

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\OlatoyosiOlamideBaba\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OlatoyosiOlamideBaba\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\OlatoyosiOlamideBaba\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package wordnet is already up-to-date!


True

In [118]:
data = {
    "intents": [
        {
            "tag": "age",
            "patterns": ["how old are you?"],
            "responses": ["I am 2 years old and my birthday was yesterday"],
        },
        {
            "tag": "greeting",
            "patterns": ["Hi", "Hello", "Hey"],
            "responses": ["Hi there", "Hello", "Hi :)"],
        },
        {
            "tag": "goodbye",
            "patterns": ["bye", "later"],
            "responses": ["Bye", "take care"],
        },
        {
            "tag": "name",
            "patterns": ["what's your name?", "who are you?"],
            "responses": [
                "I have no name yet," "You can give me one, and I will appreciate it"
            ],
        },
        {
            "tag": "coach",
            "patterns": ["Who is our data coach?", "Who is our Associate coach"],
            "responses": ["Data Coach, Associate coach"]
        },
        {
            "tag": "task",
            "patterns": ["What is my task?", "show me a task"],
            "responses": ["Data Coach, Associate coach"]
        },
        {
            "tag": "tomorrow",
            "patterns": ["What is my task tomorrow?", "show me a task next day", "What are we covering in class tomorrow?","tomorrow",],
            "responses": ["Data Coach, Associate coach"]
        },
         {
            "tag": "schedule",
            "patterns": ["What is my task tomorrow?", "show me a task next day", "What are we covering in class tomorrow?"],
            "responses": ["Data Coach, Associate coach"]
        },
        {
            "tag": "monday",
            "patterns": ["what am i doing on monday?","What is on monday","Tasks I need to complete on monday","Tasks for monday","Schedule for monday","Upcoming monday tasks"],
            "responses": ["Data Coach, Associate coach"]
        },
        
    ]
}

In [119]:
lm = WordNetLemmatizer()  # for getting words
# lists
ourClasses = []
newWords = []
documentX = []
documentY = []
# Each intent is tokenized into words and the patterns and their associated tags are added to their respective lists.
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        ournewTkns = nltk.word_tokenize(pattern)  # tokenize the patterns
        newWords.extend(ournewTkns)  # extends the tokens
        documentX.append(pattern)
        documentY.append(intent["tag"])

    if (
        intent["tag"] not in ourClasses
    ):  # add unexisting tags to their respective classes
        ourClasses.append(intent["tag"])

newWords = [
    lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation
]  # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))  # sorting words
ourClasses = sorted(set(ourClasses))  # sorting classes

trainingData = []  # training list array
outEmpty = [0] * len(ourClasses)
# bow model
for idx, doc in enumerate(documentX):
    bagOfwords = []
    text = lm.lemmatize(doc.lower())
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(
    trainingData, dtype=object
)  # coverting our data into an array afterv shuffling

x = num.array(list(trainingData[:, 0]))  # first trainig phase
y = num.array(list(trainingData[:, 1]))  # second training phase

iShape = (len(x[0]),)
oShape = len(y[0])
# parameter definition
ourNewModel = Sequential()
# In the case of a simple stack of layers, a Sequential model is appropriate

# Dense function adds an output layer
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))
# The activation function in a neural network is in charge of converting the node's summed weighted input into activation of the node or output for the input in question
ourNewModel.add(Dropout(0.5))
# Dropout is used to enhance visual perception of input neurons
ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation="softmax"))
# below is a callable that returns the value to be used with no arguments
md = tensorF.keras.optimizers.legacy.Adam(learning_rate=0.01, decay=1e-6)
# Below line improves the numerical stability and pushes the computation of the probability distribution into the categorical crossentropy loss function.
ourNewModel.compile(loss="categorical_crossentropy", optimizer=md, metrics=["accuracy"])
# Output the model in summary
print(ourNewModel.summary())
# Whilst training your Nural Network, you have the option of making the output verbose or simple.
ourNewModel.fit(x, y, epochs=200, verbose=1)
# By epochs, we mean the number of times you repeat a training set.

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 128)               5504      
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_22 (Dense)            (None, 64)                8256      
                                                                 
 dropout_15 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 9)                 585       
                                                                 
Total params: 14,345
Trainable params: 14,345
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/200
1/1 [============================

In [120]:
def ourText(text):
    newtkns = nltk.word_tokenize(text)
    newtkns = [lm.lemmatize(word) for word in newtkns]
    return newtkns


def wordBag(text, vocab):
    newtkns = ourText(text)
    bagOwords = [0] * len(vocab)
    for w in newtkns:
        for idx, word in enumerate(vocab):
            if word == w:
                bagOwords[idx] = 1
    return num.array(bagOwords)


def Pclass(text, vocab, labels):
    bagOwords = wordBag(text, vocab)
    ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
    newThresh = 0.2
    yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

    yp.sort(key=lambda x: x[1], reverse=True)
    newList = []
    for r in yp:
        newList.append(labels[r[0]])
    return newList


def getRes(firstlist, fJson):
    tag = firstlist[0]
    listOfIntents = fJson["intents"]
    for i in listOfIntents:
        if i["tag"] == tag:
            ourResult = random.choice(i["responses"])
            break
    return ourResult


In [121]:
# while True:
#     newMessage = input("")
#     intents = Pclass(newMessage, newWords, ourClasses)
#     ourResult = getRes(intents, data)
#     print(ourResult)

In [122]:
import pandas as pd

df = pd.read_excel("Schedule_Cleaned.xlsm")
df

,Date,Day,Coach/Associate,Name,Week,Cohort,Task,AM,PM,EOD
0,2022-11-28,Monday,Coach,NaN,W12,C6,Final Project,Waseem Career Talks 11-12,NaN,NaN
1,2022-11-29,Tuesday,Coach,NaN,W12,C6,Final Project,NaN,NaN,NaN
2,2022-11-30,Wednesday,Coach,NaN,W12,C6,Academy + (1),NaN,NaN,NaN
3,2022-12-01,Thursday,Coach,NaN,W12,C6,Academy + (1),NaN,NaN,NaN
4,2022-12-02,Friday,Coach,NaN,W12,C6,Academy + (1),NaN,NaN,NaN
5,2022-11-28,Monday,Associate,Nathan,W12,C6,Final Project,Dev,Project,Admin
6,2022-11-29,Tuesday,Associate,Nathan,W12,C6,Final Project,Project,Project,Admin
7,2022-11-30,Wednesday,Associate,Nathan,W12,C6,Academy + (1),Set on Academy Portfolio Building,Dev,Admin
8,2022-12-01,Thursday,Associate,Nathan,W12,C6,Academy + (1),Dev,Dev,Admin
9,2022-12-02,Friday,Associate,Nathan,W12,C6,Academy + (1),C7 Presentation,C7 Presentation,Admin


In [123]:
intents = Pclass("What are doing tomorrow?", newWords, ourClasses)
intents

1/1 [==============================] - 0s 47ms/step


['tomorrow']

In [139]:
if intents[0] == 'tomorrow':
    display(df[["Coach/Associate", "Name"]])

if intents[0] == "tomorrow":
    # date2 = datetime.date.today() + datetime.timedelta(days=1)
    # print(date2)
    date1 = datetime.date.today() - datetime.timedelta(days=11)
    Task1 = df.query('Date == @date1')['Task'].to_list()
    CoachAssociate = df.query('Date == @date1')['Coach/Associate'].to_list()
    Name1 = df.query('Date == @date1')['Name'].to_list()
    while True:
        Name =  input("Please input the Coach or Associates Coachs Name.")
        if Name not in ('Nathan','NaN'):
            print("Enter either this names exactly, Nathan or NaN")
        
        else:
            if Name == "Nathan":
                print("Tomorrow, {}, {}, will be teaching {}.".format(CoachAssociate[1],Name1[1],Task1[1]))
                print(CoachAssociate)
                print(Task1[1])
        
            if Name == "NaN":
                print("Tomorrow, {}, {}, will be teaching {}.".format(CoachAssociate[0],Name1[0],Task1[0]))
            break

    #TODO what is the poandas stuff i need to answer the question 
#     pass
# if intents =="another task":
#     pass
#     # TODO more pandas stuff

,Coach/Associate,Name
0,Coach,NaN
1,Coach,NaN
2,Coach,NaN
3,Coach,NaN
4,Coach,NaN
5,Associate,Nathan
6,Associate,Nathan
7,Associate,Nathan
8,Associate,Nathan
9,Associate,Nathan


IndexError: list index out of range

In [125]:
intents2 = Pclass("What are we doing on monday?", newWords, ourClasses)
intents2

1/1 [==============================] - 0s 34ms/step


['monday']

In [140]:
if intents2[0] == 'monday':
    display(df[["Coach/Associate", "Name"]])

if intents2[0] == "monday":
    # date2 = datetime.date.today() + datetime.timedelta(days=1)
    # print(date2)
    date2 = datetime.date.today() + datetime.timedelta(days=-datetime.date.today().weekday(), weeks=1)-datetime.timedelta(days=21)
    print(date2)
    Task1 = df.query('Date == @date2')['Task'].to_list()
    CoachAssociate = df.query('Date == @date2')['Coach/Associate'].to_list()
    Name1 = df.query('Date == @date2')['Name'].to_list()
    am1 =  df.query('Date == @date2')['AM'].to_list()
    pm1 =  df.query('Date == @date2')['PM'].to_list()
    eod = df.query('Date == @date2')['EOD'].to_list()
    while True:
        Name =  input("Please input the Coach or Associates Coachs Name.")
        if Name not in ('Nathan','NaN'):
            print("Enter either this names exactly, Nathan or NaN")
        
        else:
            if Name == "Nathan":
                print("On Monday {}, {}, {}, will be teaching {} in the morning, {} in the afternoon and {} at the end of the day.".format(date2,CoachAssociate[1],Name1[1],am1[1],pm1[1],eod[1]))
                print(CoachAssociate)
                print(Task1[1])
        
            if Name == "NaN":
                print("Tomorrow, {}, {}, will be teaching {}.".format(CoachAssociate[0],Name1[0],Task1[0]))
            break

    #TODO what is the poandas stuff i need to answer the question 
#     pass
# if intents =="another task":
#     pass
#     # TODO more pandas stuff

,Coach/Associate,Name
0,Coach,NaN
1,Coach,NaN
2,Coach,NaN
3,Coach,NaN
4,Coach,NaN
5,Associate,Nathan
6,Associate,Nathan
7,Associate,Nathan
8,Associate,Nathan
9,Associate,Nathan


2022-11-28
On Monday 2022-11-28, Associate, Nathan, will be teaching Dev in the morning, Project in the afternoon and Admin at the end of the day.
['Coach', 'Associate']
Final Project
